In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go

import lgchimera.general as general
from lgchimera.io import read_lidar_bin, read_gt

Read lidar points

In [ ]:
def read_lidar_bin(binpath):
    """Read bin files containing LiDAR point cloud data 

    Parameters
    ----------
    binpath : str
        Path to folder containing bin files

    Returns
    -------
    PC_data : list of np.array (n_pts x 3)
        List of point clouds for each frame

    """
    PC_data = []
    size_float = 4
    for i in range(len(os.listdir(binpath))):
        list_pcd = []
        bf = binpath+"/{i}.bin".format(i=str(i).zfill(10))

        try: 
            with open(bf, "rb") as f: 
                byte = f.read(size_float*4)
                while byte:
                    x, y ,z, intensity = struct.unpack("ffff", byte)
                    list_pcd.append([x, y, z])
                    byte = f.read(size_float * 4)
            np_pcd = np.asarray(list_pcd)
        except FileNotFoundError:
            print(f"file {i} wasn't found")
        
        PC_data.append(np_pcd)
    
    return PC_data

In [ ]:
binpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_26_drive_0009_sync', 'velodyne_points', 'data')
PC_data = read_lidar_bin(binpath)

In [ ]:
len(PC_data[0])

In [ ]:
fig = go.Figure(data=general.pc_plot_trace(PC_data[0], size=2))
fig.update_layout(width=1600, height=900, 
    scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

Ground truth trajectory

In [ ]:
gtpath = os.path.join(os.getcwd(), 'oxts', 'data')

In [ ]:
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_26_drive_0009_sync', 'oxts', 'data')

In [ ]:
gt_data = read_gt(gtpath)

In [ ]:
lla = gt_data[:,:3]

In [ ]:
fig = go.Figure(data=go.Scatter(x=lla[:,0], y=lla[:,1]))
fig.update_layout(width=900, height=700, 
    scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

LiDAR odometry with ICP

In [ ]:
import open3d as o3d
import time

In [ ]:
def initalize_source_and_target(source_data, target_data):
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(source_data)
    source.estimate_normals()
    source.orient_normals_towards_camera_location()

    target = o3d.geometry.PointCloud()
    target.points = o3d.utility.Vector3dVector(target_data)
    target.estimate_normals()
    target.orient_normals_towards_camera_location()

    return source, target

def p2p_ICP(source, target, threshold, trans_init):
    start_time = time.time()
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    eval_time = time.time() - start_time
    return reg_p2p, eval_time

In [ ]:
source, target = initalize_source_and_target(PC_data[0], PC_data[1])

In [ ]:
trans_init = np.eye(4)
threshold = 1

evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

In [ ]:
reg_p2p, eval_time = p2p_ICP(source, target, threshold, trans_init)
print("Elapased time: ", eval_time)
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

In [ ]:
N = len(PC_data)
R_abs = np.eye(3)
t_abs = np.zeros(3)
poses = N * [None]
poses[0] = (R_abs, t_abs)

for i in range(N-1):
    print(i)
    trans_init = np.eye(4)
    threshold = 1
    source, target = initalize_source_and_target(PC_data[i+1], PC_data[i])
    reg_p2p, eval_time = p2p_ICP(source, target, threshold, trans_init)
    R_hat = reg_p2p.transformation[:3,:3]
    t_hat = reg_p2p.transformation[:3,3]

    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    poses[i] = (R_abs.copy(), t_abs.copy())
    

In [ ]:
positions = np.zeros((N,3))
for i in range(1,N):
    positions[i] = poses[i-1][1]

In [ ]:
fig = go.Figure(data=go.Scatter(x=positions[:,0], y=positions[:,1]))
fig.update_layout(width=900, height=700, 
    scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()